In [122]:
%load_ext autoreload
%autoreload 2

import os
import sys
sys.path.append("../../src")
import pandas as pd
import numpy as np
import synapseclient as sc
import utils.munging_utils as query
from datetime import datetime

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
df = query.get_walking_synapse_table(recordIds = ["cfb1f131-e49a-491f-ba53-ae9878688be7",
                                                  "0f4222f1-6ad5-45c5-bb53-ed2f53c5f3b8",
                                                  "42b1dcdf-9392-49f3-b3e0-0bac80c0e6ba",
                                                  "507a9af9-4638-4549-9613-d6089d3b32c3",
                                                 ],
                               table_id = "syn12514611",
                               version = "V2")

Querying V2 Data


In [44]:
def get_fs(path, measurement):
    try:
        data = query.open_filepath(path)[1:]
        data = data[data["sensorType"] == measurement]
        time_end = (data["timestamp"].iloc[-1])
        N = data.shape[0]
    except:
        return 0
    return N/time_end

In [29]:
data = query.open_filepath(df["walk_motion.json_pathfile"].iloc[3])

In [30]:
data[data["sensorType"] == "rotationRate"].shape[0]/data["timestamp"].iloc[-1]

3.2773978196604086

In [31]:
data[data["sensorType"] == "acceleration"].shape[0]/data["timestamp"].iloc[-1]

31.777342602321387

In [45]:
df["acceleration_sampling_rate"] = df["walk_motion.json_pathfile"].apply(get_fs, measurement = "acceleration")
df["gyro_sampling_rate"] = df["walk_motion.json_pathfile"].apply(get_fs, measurement = "rotationRate")

In [47]:
df[["recordId", "healthCode", "phoneInfo", "appVersion", "acceleration_sampling_rate", "gyro_sampling_rate"]]

,recordId,healthCode,phoneInfo,appVersion,acceleration_sampling_rate,gyro_sampling_rate
0,0f4222f1-6ad5-45c5-bb53-ed2f53c5f3b8,6d9b0436-6740-4c50-81ea-4696647b49fb,LGE LM-Q710.FG,"version 0.14.0-internal, build 233",49.786232,4.997025
1,42b1dcdf-9392-49f3-b3e0-0bac80c0e6ba,6d9b0436-6740-4c50-81ea-4696647b49fb,LGE LM-Q710.FG,"version 0.14.0-internal, build 233",49.799485,5.008480
2,507a9af9-4638-4549-9613-d6089d3b32c3,db8e5408-b39e-4dbb-902e-fa0131522469,Samsung SM-J320H,"version 0.14.0-internal, build 233",5.263171,0.000000
3,cfb1f131-e49a-491f-ba53-ae9878688be7,6d9b0436-6740-4c50-81ea-4696647b49fb,LGE LM-Q710.FG,"version 0.14.0-internal, build 233",31.758177,3.285600


In [3]:
data = query.get_file_entity("syn21281633")

In [10]:
data["gender_annot"] = data["gender"].fillna(-1)

In [16]:
data[data["healthCode"] == "000240d1-1110-4dd2-a2d0-e344c37efd68"][["PD", "MS", "version"]]

,PD,MS,version
14948,0.0,NaN,V1
84709,0.0,NaN,V1


In [136]:
syn = sc.login()
data = syn.tableQuery("SELECT * FROM {} where dataGroups\
                               NOT LIKE '%test_user%'".format("syn15673379")).asDataFrame()

# EMSdata = syn.tableQuery("SELECT * FROM {} where dataGroups\
#                                NOT LIKE '%test_user%'".format("syn10295288")).asDataFrame()

# data = syn.tableQuery("SELECT * FROM {} where dataGroups\
#                                NOT LIKE '%test_user%'".format("syn15673379")).asDataFrame()

Welcome, aryton tediarjo!



In [141]:
data[data["healthCode"] == "33466738-a918-4cf2-8959-96fffa91c90c"]["dataGroups"].iloc[0]

'clinical_consent,gr_BR_II,gr_DT_T,gr_SC_DB,gr_ST_T,parkinsons'

In [137]:
data   = data[data["diagnosis"] != "no_answer"] 
data["PD"] = data["diagnosis"].map({"parkinsons":1, "control":0})
data["age"] = data["birthYear"].apply(lambda year: datetime.now().year - year)

In [138]:
# for each healthCode, collect the latest app Version, 
# choose most recent (or more frequent appVersion, choose most recent for now)

dict_ = {}
dict_["healthCode"] = []
dict_["gender"]     = []
dict_["age"]        = []
dict_["appVersion"] = []
dict_["phoneInfo"]  = []
dict_["version"]    = []
dict_["PD"]         = []
dict_["MS"]         = []

for healthCode in ["33466738-a918-4cf2-8959-96fffa91c90c", "debb8d54-435c-44ff-8684-5bc05cc407b7"]:
    temp = data[data["healthCode"] == healthCode]
    gender_status = [gender for gender in temp["sex"].unique()\
                   if (gender == "male") or (gender == "female")]
    print(gender_status)
    age_status = temp["age"].max()
    
    if "PD" in temp.columns:
        PD_status = temp["PD"].max()
        MS_status = 0 
    elif "MS" in temp.columns:
        PD_status = 0
        MS_status = temp["PD"].max()
    
    temp = temp[temp["createdOn"] == temp["createdOn"].max()]
    
    
    
    
    

['female']


TypeError: append() takes exactly one argument (0 given)

In [96]:
[gender for gender in ["female", -1] if (gender == "male") or (gender == "female")]

In [251]:
### version 1 

demo_data = syn.tableQuery("SELECT age, healthCode, inferred_diagnosis as PD, gender FROM {} where dataGroups\
                               NOT LIKE '%test_user%'".format("syn10371840")).asDataFrame()
demo_data = demo_data[(demo_data["gender"] == "Female") | (demo_data["gender"] == "Male")]
demo_data = demo_data.dropna(subset = ["PD"], thresh = 1)                     ## drop if no diagnosis
demo_data["PD"] = demo_data["PD"].map({True :1.0, False:0.0})                 ## encode as numeric binary
demo_data["age"] = demo_data["age"].apply(lambda x: float(x))                 ## convert age to float
demo_data = demo_data[(demo_data["age"] <= 100) & (demo_data["age"] >= 10)]   ## subset to realistic age ranges
demo_data["gender"] = demo_data["gender"].apply(lambda x: x.lower())          ## lowercase gender for consistencies
demo_data_V1 = demo_data.reset_index(drop = True)

In [252]:
demo_data_V1.groupby("healthCode").nunique()["gender"].mean()

1.0

In [253]:
### version 2

demo_data = syn.tableQuery("SELECT birthYear, createdOn, healthCode, diagnosis as PD, sex as gender FROM {} where dataGroups\
                               NOT LIKE '%test_user%'".format("syn15673379")).asDataFrame()
demo_data = demo_data[(demo_data["gender"] == "male") | (demo_data["gender"] == "female")]
demo_data   = demo_data[demo_data["PD"] != "no_answer"]               
demo_data["PD"] = demo_data["PD"].map({"parkinsons":1, "control":0})
demo_data["age"] = pd.to_datetime(demo_data["createdOn"], unit = "ms").dt.year - demo_data["birthYear"]
demo_data = demo_data[(demo_data["age"] <= 100) & (demo_data["age"] >= 10)]
demo_data = demo_data.drop(["birthYear", "createdOn"], axis = 1)
demo_data_v2 = demo_data.reset_index(drop = True)

In [254]:
demo_data_V2.groupby("healthCode").nunique()["gender"].mean()

1.0

In [255]:
### version ems
demo_data = syn.tableQuery("SELECT healthCode, dataGroups as MS, 'demographics.gender' as gender,\
                            'demographics.age' as age FROM {}\
                            where dataGroups NOT LIKE '%test_user%'".format("syn10235463")).asDataFrame()
demo_data = demo_data[(demo_data["gender"] == "Male") | (demo_data["gender"] == "Female")]
demo_data["gender"] = demo_data["gender"].apply(lambda x: x.lower())
demo_data = demo_data[(demo_data["age"] <= 100) & (demo_data["age"] >= 10)]
demo_data["MS"] = demo_data["MS"].map({"ms_patient":1, "control":0})
demo_data_EMS = demo_data.reset_index(drop = True)

In [256]:
demo_data_EMS.groupby("healthCode").nunique()["gender"].mean()

1.0

In [262]:
df = pd.concat([demo_data_V1, demo_data_V2, demo_data_EMS], sort = False).reset_index(drop = True)

In [269]:
df["PD"] = df["PD"].fillna(0)
df["MS"] = df["MS"].fillna(0)

In [270]:
df

,age,healthCode,PD,gender,MS
0,33.0,000240d1-1110-4dd2-a2d0-e344c37efd68,0.0,male,0.0
1,35.0,0005a31d-e52c-447c-9971-ccc7bef667fb,0.0,male,0.0
2,70.0,00081bd9-9abd-4003-b035-de6cc3e8c922,0.0,male,0.0
3,23.0,00086114-0bb3-460e-8841-94bc35d27d71,0.0,female,0.0
4,71.0,001702e9-908d-4419-9c08-8ef5615d6b67,1.0,male,0.0
5,21.0,00182372-b75c-48f0-a74d-2c1f447bb0bd,0.0,male,0.0
6,43.0,00185644-b15c-4648-bc0d-1bffce96efeb,0.0,male,0.0
7,34.0,0019f348-12ec-442b-b5a3-a6df67227cbf,0.0,male,0.0
8,18.0,0024337d-e161-413b-8e62-179b7b86b019,0.0,male,0.0
9,30.0,0025250a-24ac-45ea-befa-c83a969c6ab2,0.0,male,0.0
